In [191]:
from struct import unpack
import numpy as np
import matplotlib.pylab as plt
import gzip
from scipy.stats import multivariate_normal

def loadmnist(imagefile, labelfile):

    # Open the images with gzip in read binary mode
    images = gzip.open(imagefile, 'rb')
    labels = gzip.open(labelfile, 'rb')

    # Get metadata for images
    images.read(4)  # skip the magic_number
    number_of_images = images.read(4)
    number_of_images = unpack('>I', number_of_images)[0]
    rows = images.read(4)
    rows = unpack('>I', rows)[0]
    cols = images.read(4)
    cols = unpack('>I', cols)[0]

    # Get metadata for labels
    labels.read(4)
    N = labels.read(4)
    N = unpack('>I', N)[0]

    # Get data
    x = np.zeros((N, rows*cols), dtype=np.uint8)  # Initialize numpy array
    y = np.zeros(N, dtype=np.uint8)  # Initialize numpy array
    for i in range(N):
        for j in range(rows*cols):
            tmp_pixel = images.read(1)  # Just a single byte
            tmp_pixel = unpack('>B', tmp_pixel)[0]
            x[i][j] = tmp_pixel
        tmp_label = labels.read(1)
        y[i] = unpack('>B', tmp_label)[0]

    images.close()
    labels.close()
    return (x, y)

def displaychar(image):
    plt.imshow(np.reshape(image, (28,28)), cmap=plt.cm.gray)
    plt.axis('off')
    plt.show()

In [192]:
trainImage, trainLabel = loadmnist('train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz')

In [193]:
# divide into training set of size 50000 and a seperate validation set of size 10000
training_indices = range(0, 50000)
validation_indices = range(50000, 60000)
class_probabilities_for_training = [0] * 10
image_class = []

trainx = trainImage[training_indices] 
trainy = trainLabel[training_indices]
validx = trainImage[validation_indices]
validy = trainLabel[validation_indices]

# Getting all labels into digit x classes
for i in range(10):
    image_class.append([])
for (a,b) in zip(trainx, trainy):
    image_class[b].append(a)

# get pi for each class
for i in range(len(trainy)):
    class_probabilities_for_training[trainy[i]] += 1

# getting the probability of it
for j in range(len(class_probabilities_for_training)):
    class_probabilities_for_training[j] /= float(50000)   

In [194]:
cov_class = [[]] * 10
mean_class = [[]] * 10
normalize_class = [[]] * 10

# get covariance and mean matrix for each class
for a in range(10):
    cov_class[a] = np.cov(image_class[a], rowvar=False)
    mean_class[a] = np.mean(image_class[a], axis=0)

(784, 784)


In [195]:
# add cI on to covariance matrix
temp = []
# making identity matrix
temp_identity = np.identity(784)
# defining c
c = 4000

# multiplying by c
for a in range(len(temp_identity)):
    for b in range(len(temp_identity[a])):
        temp_identity[a][b] *= c
#print(temp_identity)
for c in range(0, 10):
    # np.add
    cov_class[c] = np.add(cov_class[c],temp_identity)

In [196]:
# get gaussian value for each class
normalize_class = []
for i in range(10):
    normalize_class.append(multivariate_normal(mean=mean_class[i], cov=cov_class[i]))

In [197]:
import math
lt = []
normalize_whole = []
predictions = []
for i in range(10000):
    lt.append([])
    
for j in range(10):
    normalize_whole.append(lt)

# add log pi and probability and get the prediction
for b in validx:
    prob = []
    for y in range(10):
        prob.append(math.log(class_probabilities_for_training[y])+normalize_class[y].logpdf(b))
    index = prob.index(max(prob))
    predictions.append(index)
print prob[0].shape

In [198]:
print(predictions)
print(validy)
test_error = []
# get error index
for i in range(len(predictions)):
    if predictions[i] != validy[i]:
        test_error.append(i)
        
    

[3, 8, 6, 9, 6, 4, 5, 3, 8, 4, 5, 2, 3, 8, 4, 8, 1, 5, 0, 5, 9, 7, 4, 1, 0, 3, 0, 6, 2, 9, 9, 4, 1, 3, 6, 8, 0, 7, 7, 6, 8, 9, 0, 3, 8, 3, 7, 9, 8, 4, 4, 1, 2, 9, 8, 1, 1, 0, 6, 6, 5, 0, 1, 1, 7, 2, 7, 3, 1, 4, 0, 5, 0, 6, 9, 7, 6, 8, 8, 9, 4, 0, 6, 1, 9, 2, 6, 3, 1, 4, 4, 5, 6, 6, 1, 7, 2, 8, 6, 9, 7, 0, 9, 1, 6, 2, 8, 3, 6, 4, 9, 5, 8, 6, 8, 7, 8, 8, 6, 9, 1, 7, 6, 0, 9, 6, 7, 0, 9, 7, 1, 3, 6, 8, 9, 6, 1, 7, 5, 1, 3, 3, 5, 7, 9, 9, 6, 7, 3, 4, 1, 0, 4, 2, 4, 5, 0, 0, 1, 6, 6, 4, 7, 9, 4, 6, 5, 2, 6, 9, 8, 8, 8, 5, 9, 3, 8, 9, 1, 8, 8, 3, 4, 4, 3, 0, 4, 5, 4, 4, 1, 8, 0, 6, 1, 3, 2, 0, 8, 6, 0, 3, 5, 4, 9, 0, 3, 1, 0, 9, 3, 2, 8, 3, 3, 7, 4, 9, 2, 1, 6, 2, 1, 5, 7, 1, 9, 7, 9, 2, 2, 8, 1, 7, 7, 0, 1, 1, 8, 0, 0, 6, 6, 4, 7, 9, 7, 2, 9, 1, 5, 2, 5, 3, 7, 7, 0, 0, 8, 2, 3, 1, 3, 5, 1, 3, 6, 4, 8, 7, 6, 2, 8, 1, 8, 6, 6, 8, 7, 5, 6, 0, 4, 8, 4, 9, 3, 2, 3, 6, 2, 0, 1, 1, 6, 2, 5, 3, 0, 4, 3, 5, 8, 6, 9, 7, 8, 8, 3, 9, 0, 0, 3, 1, 4, 2, 5, 2, 9, 4, 5, 5, 5, 6, 0, 7, 1, 8, 0, 7, 3, 0, 5, 

In [202]:
# error ratemak
print(len(test_error) / float(len(validy)))

0.0414


In [203]:
print test_error

[47, 74, 78, 88, 134, 178, 186, 212, 236, 239, 246, 317, 329, 340, 342, 349, 355, 359, 369, 370, 375, 396, 414, 417, 426, 431, 446, 514, 522, 564, 572, 618, 632, 639, 644, 698, 714, 728, 740, 753, 768, 802, 841, 858, 881, 897, 908, 926, 946, 1004, 1058, 1100, 1130, 1158, 1164, 1172, 1180, 1200, 1230, 1238, 1248, 1278, 1280, 1298, 1300, 1314, 1315, 1322, 1343, 1346, 1380, 1414, 1476, 1522, 1530, 1544, 1572, 1656, 1698, 1740, 1759, 1764, 1794, 1803, 1944, 1964, 1986, 1987, 1988, 1990, 2014, 2022, 2070, 2085, 2086, 2122, 2129, 2138, 2144, 2146, 2166, 2172, 2173, 2210, 2218, 2241, 2246, 2272, 2275, 2279, 2280, 2290, 2294, 2316, 2324, 2390, 2424, 2452, 2550, 2582, 2600, 2671, 2674, 2699, 2707, 2708, 2738, 2762, 2800, 2802, 2808, 2812, 2828, 2862, 2873, 2875, 2884, 2886, 2894, 2899, 2907, 2910, 2914, 2928, 2932, 2938, 2953, 2962, 2975, 2981, 3013, 3015, 3017, 3024, 3029, 3036, 3052, 3113, 3116, 3148, 3156, 3196, 3216, 3244, 3264, 3316, 3398, 3417, 3470, 3476, 3478, 3482, 3507, 3552, 3558, 35

In [174]:
last_question = []
for b in range(10):
    prob = []
    for y in range(10):
        prob.append(math.log(class_probabilities_for_training[y]) + normalize_class[y].logpdf(test_error[b]))
    last_question.append(prob)

In [204]:
print last_question

[[-4176.3017609774679, -4168.2612798359251, -4183.6164271076959, -4179.4867165129508, -4182.6630143640887, -4176.2338125417655, -4176.6811104641538, -4172.0972153117609, -4188.8678017517268, -4174.6789540752343], [-4336.7929474496714, -4376.6618633436756, -4333.3323502533931, -4335.9628149853324, -4351.53961888098, -4330.4260258742052, -4346.077358814131, -4345.191774738757, -4349.3386359060269, -4346.8935400166265], [-4366.9168358815241, -4415.939794260963, -4361.4903148069088, -4365.4269623347946, -4383.2239220855381, -4359.4188101872696, -4377.9330533711554, -4377.6871558329758, -4379.5278060208266, -4379.2726730312388], [-4449.392994789102, -4523.6227494224649, -4438.6343866001052, -4446.1804794051141, -4469.9602976548076, -4438.8437474446091, -4465.2044959613186, -4466.6614470562863, -4462.244278472559, -4467.9722841424546], [-4960.6457817981918, -5193.5458959417665, -4917.6816683819097, -4948.1605939472083, -5007.4189883383615, -4931.9492259624276, -5007.0857259119175, -5018.2826

In [201]:
print(prob.shape)

()
